# Writing files

In [1]:
from your import Your, Writer
import os
import numpy as np
import tempfile
from urllib.request import urlretrieve

In [2]:
import logging

logger = logging.getLogger()
logger = logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(threadName)s - %(levelname)s -" " %(message)s",
)

We can write + convert files using the [Writer](https://thepetabyteproject.github.io/your/writer/#writer) class in [Your](https://thepetabyteproject.github.io/your/your/#your). Let's read a Filterbank file and write out a PSRFITS and a Filterbank file

In [3]:
temp_dir = tempfile.TemporaryDirectory()
download_path = str(temp_dir.name) + "/FRB180417.fil"
url = "https://zenodo.org/record/3905426/files/FRB180417.fil"
urlretrieve(
    url, download_path,
)
fil_file = download_path
your_original_file = Your(fil_file)

While writing files you can also select the required channel and spectra range. Just give the required ranges as input to the Writer class.  

In [4]:
writer_object = Writer(
    your_original_file,
    nstart=0,
    nsamp=10,
    c_min=10,
    c_max=100,
    outdir=".",
    outname="filfromfil",
)

## Writing Filterbanks

To write a filterbank we use the [`to_fil`](https://thepetabyteproject.github.io/your/writer/#to_fil) method,

In [5]:
writer_object.to_fil()

Output()

We can just read this Filterbank using [Your](https://thepetabyteproject.github.io/your/your/#your) to make sure that the header parameters and the data are correctly written.

In [6]:
your_written_filterbank = Your("filfromfil.fil")
your_written_filterbank.your_header

Unified Header:
  "basename": "filfromfil"
  "bw": -90.0
  "center_freq": 1410.0
  "dec_deg": 13.964475555555554
  "dtype": "uint8"
  "fch1": 1455.0
  "filelist": [
    "filfromfil.fil"
  ]
  "filename": "filfromfil.fil"
  "foff": -1.0
  "format": "fil"
  "frequency_decimation_factor": 1
  "gb": 75.61933243565039
  "gl": 97.96004079061271
  "native_foff": -1.0
  "native_nchans": 90
  "native_nspectra": 10.0
  "native_tsamp": 0.00126646875
  "nbits": 8
  "nchans": 90
  "npol": 1
  "nspectra": 10
  "ra_deg": 186.65681708333335
  "source_name": "src1"
  "time_decimation_factor": 1
  "tsamp": 0.00126646875
  "tstart": 58682.620316710374
  "tstart_utc": "2019-07-18T14:53:15.364"

We can also compare data from both the original file and the newly written file:

In [7]:
original_data = your_original_file.get_data(0, 10)[:, 10:100]
filterbank_data = your_written_filterbank.get_data(0, 10)

In [8]:
np.array_equal(original_data, filterbank_data)

True

Looks good!

## Writing PSRFITS



All the inputs defined while declaring Writer are still gonna be used. Let's change the outname for clarity
and select some channels

In [9]:
writer_object = Writer(
    your_original_file,
    nstart=0,
    nsamp=10,
    c_min=0,
    c_max=100,
    outdir=".",
    outname="fitsfromfil",
)

In [10]:
writer_object.outname = "fitsfromfil"
writer_object.c_min = 0
writer_object.c_max = 100

To write a PSRFITS we use the [`to_fits`](https://thepetabyteproject.github.io/your/writer/#to_fits) method,

In [11]:
writer_object.to_fits()

2020-11-02 13:29:04,530 - root - MainThread - INFO - ObsInfo updated with relevant parameters
2020-11-02 13:29:04,531 - your.formats.fitswriter - MainThread - INFO - Setting the following info to be written in ./fitsfromfil.fits 
 {
    "ant_x": -1601185.63,
    "ant_y": -5041978.15,
    "ant_z": 3554876.43,
    "bmaj_deg": 0.0,
    "bmin_deg": 0.0,
    "bpa_deg": 0.0,
    "bw": -100.0,
    "chan_bw": -1.0,
    "dec_str": "13:57:52.1120",
    "dt": 0.00126646875,
    "fcenter": 1415.0,
    "file_date": "2020-11-02T18:29:04",
    "longitude": -107.61833443381258,
    "nbits": 8,
    "nchan": 100,
    "nsblk": 10,
    "nsuboffs": 0.0,
    "obs_date": "2019-07-18T14:53:15",
    "observer": "Human",
    "proj_id": "Awesome_Project",
    "ra_str": "12:26:37.6361",
    "scan_len": 0.0126646875,
    "src_name": "src1",
    "stt_imjd": 58682,
    "stt_lst": 12438.193191755068,
    "stt_offs": 0.3637763159349561,
    "stt_smjd": 53595,
    "telescope": "VLA"
}
2020-11-02 13:29:04,542 - root - M

Output()

2020-11-02 13:29:04,701 - your.writer - MainThread - INFO - Writing data to ./fitsfromfil.fits from subint = 0 to 1.


2020-11-02 13:29:05,464 - your.writer - MainThread - INFO - All spectra written to ./fitsfromfil.fits


Similar to reading the PSRFITS file, we can also read a filterbank file and convert to both formats by first defining a [`writer_object`](https://thepetabyteproject.github.io/your/writer/) and using the [`to_fits`](https://thepetabyteproject.github.io/your/writer/#to_fits) and [`to_fil`](https://thepetabyteproject.github.io/your/writer/#to_fil) methods.